In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.mixture import GaussianMixture
from sklearn.manifold import MDS, Isomap
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import DBSCAN

from scipy.special import kl_div
import MDAnalysis as mda
import sys
import itertools
from scipy.special import rel_entr
import matplotlib.pyplot as plt
import glob
import numpy as np
import itertools
import pickle
from scipy.spatial.distance import pdist, cdist


import keras
from keras import layers
from tensorflow.keras.models import Model
import tensorflow as tf
from scikeras.wrappers import KerasClassifier, KerasRegressor

2023-04-25 17:50:11.114774: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 17:50:12.034517: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Making test data

selection = "name CA"
data_dict = {}

for system in glob.glob("../binding_spots_project/gpcr_sampling/b2ar/b2ar_centered_aligned/*"):
    name = system.split("/")[-1]
    cosmos = mda.Universe(glob.glob(f"{system}/*gro")[0], glob.glob(f"{system}/*xtc"))
    size = cosmos.select_atoms(selection).positions.flatten()
    pos = np.zeros(size.reshape(1, size.shape[0]).shape)
    for ts in cosmos.trajectory:
        pos = np.concatenate((pos, cosmos.select_atoms(selection).positions.reshape(1, pos.shape[1])))
    pos = pos[~np.all(pos == 0, axis=1)]
    data_dict[name] = pos
    
with open('data_dict.pkl', 'wb') as f:
    pickle.dump(data_dict, f)
    

In [3]:
with open('data_dict.pkl', 'rb') as handle:
    data_dict = pickle.load(handle)


In [4]:
system1, system2 = "popc", "chol-site-3"

X = np.concatenate([v for k, v in data_dict.items() if k == system1 or k == system2])
Y = np.concatenate([[-1 for i in range(data_dict[system1].shape[0])], [1 for i in range(data_dict[system2].shape[0])]])


## Autoencoder 

In [6]:
"""device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device.")"""

'device = (\'cuda\' if torch.cuda.is_available() else \'cpu\')\nprint(f"Using {device} device.")'

In [7]:

def create_model(optimizer='adam', kernel_initializer='glorot_uniform'):
    model = tf.keras.Sequential()
    model.add(layers.Dense(64,activation='relu',kernel_initializer=kernel_initializer))
    model.add(layers.Dense(1,activation='sigmoid',kernel_initializer=kernel_initializer))

    model.compile(loss="MeanAbsoluteError",optimizer=optimizer, metrics=["accuracy"])

    return model


m = create_model()
print(type(m))



<class 'keras.engine.sequential.Sequential'>


2023-04-25 17:50:35.165372: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 17:50:35.284874: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-25 17:50:35.285090: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [13]:
X = np.random.random((3,2))
Y = np.array([0, 1, 0])

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(X.shape[1],)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='relu'))

model.compile(loss="mse")
print(model.fit(X, Y))

2023-04-25 17:57:51.491536: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:429] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2023-04-25 17:57:51.491594: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:438] Possibly insufficient driver version: 510.108.3
2023-04-25 17:57:51.513340: E tensorflow/compiler/xla/status_macros.cc:57] INTERNAL: RET_CHECK failure (tensorflow/compiler/xla/service/gpu/gpu_compiler.cc:618) dnn != nullptr 
*** Begin stack trace ***
	tsl::CurrentStackTrace[abi:cxx11]()
	
	xla::status_macros::MakeErrorStream::Impl::GetStatus()
	xla::gpu::GpuCompiler::OptimizeHloModule(xla::HloModule*, stream_executor::StreamExecutor*, stream_executor::DeviceMemoryAllocator*, xla::gpu::GpuTargetConfig const&, xla::AutotuneResults const*)
	xla::gpu::GpuCompiler::RunHloPasses(std::unique_ptr<xla::HloModule, std::default_delete<xla::HloModule> >, stream_executor::StreamExecutor*, xla::Compiler::CompileOptions const&)
	xla::Service::BuildExecutable(xl

InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_2' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/usr/lib/python3/dist-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
      self._run_once()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 1896, in _run_once
  

In [16]:
def create_model():
    
    class autoencoder(tf.keras.Model):
        def __init__(self):
            
            super().__init__()
            
            self.encoder = tf.keras.Sequential()
            self.encoder.add(layers.Dense(1024, activation="relu", input_dim=933))
            self.encoder.add(layers.Dense(2, activation="relu", input_dim=1024))

            self.decoder = tf.keras.Sequential()
            self.decoder.add(layers.Dense(1024, activation="relu", input_dim=2))
            self.decoder.add(layers.Dense(933, activation="relu", input_dim=1024))
            
        def call(self, inputs, training=False):
            
            encoded = self.encoder(inputs)
            decoded = self.decoder(encoded)
            
            return decoded

    ae = autoencoder()
    ae.compile(loss="mse")
    
    return ae


In [17]:

m = KerasRegressor(model=create_model)
m.fit(X, X)
"""pipeline = Pipeline([
    ('clf',m)
])

pipeline.fit(X, X)"""



TypeError: object of type 'NoneType' has no len()

In [15]:
class Autoencoder(nn.Module):

    def __init__(self, in_shape=10, enc_shape=3, dimensions=[7,4], activation=nn.ReLU()):
        super(Autoencoder, self).__init__()
        self.encoder = [nn.Linear(in_shape, dimensions[0]), activation]
        self.decoder = [nn.Linear(enc_shape, dimensions[-1]), activation]
        for i in range(len(dimensions) - 1):
            self.encoder.append(nn.Linear(dimensions[i], dimensions[i+1]))
            self.encoder.append(activation)
            self.decoder.append(nn.Linear(dimensions[::-1][i], dimensions[::-1][i+1]))
            self.decoder.append(activation)
        self.encoder.append(nn.Linear(dimensions[-1], enc_shape))
        self.decoder.append(nn.Linear(dimensions[0], in_shape))
        self.encoder = nn.Sequential(*self.encoder)
        self.decoder = nn.Sequential(*self.decoder)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def score(self, x):
        return "haha"
    
autoencoder = NeuralNet(
    module = Autoencoder().double().to(device),
    criterion = nn.MSELoss()
)

## FLOW

In [16]:
n_comps_dim_reduc = [i for i in range(2, 7)] if X.shape[1] > 5 else [i for i in range(2, X.shape[1])]
n_comps_cluster = [i for i in range(2, 10)]

In [17]:
def construct_params(X):
    
    next = 2**np.ceil(np.log(X.shape[1])/np.log(2))
    to_try = [next/4, next/2, next, 2*next, 4*next]
    return to_try

print(construct_params(np.zeros((1, 16))))


[4.0, 8.0, 16.0, 32.0, 64.0]


In [18]:

default_steps = {
    "dim_reducer": {
        PCA(): {
            "n_components": n_comps_dim_reduc
        },
        "autoencoder": {
            "module__enc_shape": [i for i in range(2, 10)]
        }
    },
    "classifier": {
        GaussianMixture(): {
            "n_components": n_comps_cluster,
            "covariance_type": ["full", "spherical", "diag", "tied"]
        }
    }
}


In [19]:
autoencoder.get_params().keys()

dict_keys(['module', 'criterion', 'optimizer', 'lr', 'max_epochs', 'batch_size', 'iterator_train', 'iterator_valid', 'dataset', 'train_split', 'callbacks', 'predict_nonlinearity', 'warm_start', 'verbose', 'device', '_params_to_validate', 'callbacks__epoch_timer', 'callbacks__train_loss', 'callbacks__train_loss__name', 'callbacks__train_loss__lower_is_better', 'callbacks__train_loss__on_train', 'callbacks__valid_loss', 'callbacks__valid_loss__name', 'callbacks__valid_loss__lower_is_better', 'callbacks__valid_loss__on_train', 'callbacks__print_log', 'callbacks__print_log__keys_ignored', 'callbacks__print_log__sink', 'callbacks__print_log__tablefmt', 'callbacks__print_log__floatfmt', 'callbacks__print_log__stralign'])

In [20]:
params = default_steps["dim_reducer"]["autoencoder"]
grid = GridSearchCV(estimator=autoencoder, param_grid=params, n_jobs=-1, cv=3, scoring="accuracy")
grid_result = grid.fit(torch.Tensor(X).double().to(device))

/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py", line 529, in _fit_and_score
    estimator.fit(X_train, **fit_params)
  File "/home/eurastof/.local/lib/python3.10/site-packages/skorch/net.py", line 1230, in fit
    self.partial_fit(X, y, **fit_params)
  File "/home/eurastof/.local/lib/python3.10/site-packages/skorch/net.py", line 1189, in partial_fit
    self.fit_loop(X, y, **fit_params)
  File "/home/eurastof/.local/lib/python3.10/site-packages/skorch/net.py", line 1101, in fit_loop
    self.run_single_epoch(iterator_train, training=True, prefix="train",
  File "/home/eurastof/.local/lib/python3.10/site-packages/skorch/net.py", line 1137, in run_single_epoch
    step = step_fn(batch, **fit_params)
  File "/home/eurasto

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
def build_pipelines(step_grid):
    
    reducers = step_grid["dim_reducer"]
    classifiers = step_grid["classifier"]
    combinations = list(itertools.product(reducers.keys(), classifiers.keys()))
    pipelines = []
    
    for comb in combinations:
        print(f"fitting combination: {comb}")
        grid = {}
        for param, values in step_grid["dim_reducer"][comb[0]].items():
            key = f"dim_reducer__{param}"
            grid[key] = values
        for param, values in step_grid["classifier"][comb[1]].items():
            key = f"classifier__{param}"
            grid[key] = values
        print(f"with params {grid}")
        pipe = Pipeline(steps=[("dim_reducer", comb[0]), ("classifier", comb[1])])
        cv = GridSearchCV(pipe, param_grid=grid).fit(X)
        pipelines.append(cv.best_estimator_)

    return pipelines



In [ ]:

def caluclate_KL(pipeline, X, Y):
    
    preds = pipeline.predict(X)
    cluster_populations = []
    for system in set(Y.flatten()):
        system_preds = preds[np.where(Y == system)[0]]
        populations = [system_preds[np.where(system_preds == i)].shape[0] / system_preds.shape[0] for i in set(preds)]
        cluster_populations.append(populations)
        
    return sum(rel_entr(cluster_populations[0], cluster_populations[1])) # only two systems atm
    


In [ ]:

def flow(X, Y, step_grid):
    
    best_KL = 0
    
    pipelines = build_pipelines(step_grid=step_grid)
    best_pipe = pipelines[0]
    for pipe in pipelines:
        KL = caluclate_KL(pipe, X, Y)
        if KL > best_KL:
            best_KL = KL
            best_pipe = pipe
    
    return best_pipe, best_KL    
        


In [ ]:
data = np.load("/wrk/eurastof/somethingwithflow/drors_for_all.npy")

systems = [2,7]

X = data[(data[:,4] == systems[0]) | (data[:,4] == systems[1])]
Y = X[:,4]
X = X[:,:-1]


[[ 2.24704027  8.60608992 10.66413074  9.39539106]
 [ 2.02470636  8.13739064 12.55672955  8.39867499]
 [ 1.67873907  7.7028476  13.27813437  8.96608857]
 ...
 [ 3.67762423  7.15872565 10.17563048  7.04171149]
 [ 3.70295858  6.69223749  9.90305706  8.28288315]
 [ 3.64750528  6.3168512   9.96487556  7.330726  ]]


In [ ]:
AE = Autoencoder(in_shape=X.shape[1], enc_shape=2, dimensions=[6,4], activation=nn.ReLU())

feature selection
    
random seeds


